# Here is the code to capture frames from video. Read comment accordingly

In [ ]:
import os
import subprocess
import argparse
from pathlib import Path
from datetime import datetime

def extract_frames(video_path, output_dir, fps=1, format="jpg", quality=2, prefix="frame"):
    """
    Extract frames from a video using FFmpeg

    Args:
        video_path (str): Path to the input video file
        output_dir (str): Directory where frames will be saved
        fps (int): Frames per second to extract (default: 1)
        format (str): Image format (jpg, png)
        quality (int): JPEG quality level (1-31 where 2 is highest)
        prefix (str): Prefix for output filenames
    """
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Generate a timestamp for this extraction session
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Build FFmpeg command
    cmd = [
        "ffmpeg",
        "-i", video_path,
        "-vf", f"fps={fps}",
    ]

    # Add quality parameter for JPEG format
    if format == "jpg":
        cmd.extend(["-q:v", str(quality)])

    # Add output file and logging options
    cmd.extend([
        f"{output_dir}/{prefix}_{timestamp}%04d.{format}",
        "-loglevel", "error",
        "-stats"
    ])

    try:
        print(f"Extracting frames from {video_path}...")
        # Let FFmpeg output directly to console for progress display
        result = subprocess.run(cmd, check=True)
        print("Frame extraction completed successfully!")

        # Count extracted frames
        frame_count = len(list(Path(output_dir).glob(f"{prefix}_{timestamp}*.{format}")))
        print(f"Extracted {frame_count} frames to {output_dir}")

    except subprocess.CalledProcessError as e:
        print(f"Error extracting frames: {e}")
    except FileNotFoundError:
        print("FFmpeg not found. Please install FFmpeg and ensure it's in your PATH.")

def main():
    parser = argparse.ArgumentParser(description="Extract frames from a video file")
    parser.add_argument("video", help="Input video file path")
    parser.add_argument("-o", "--output", help="Output directory (default: frames_<filename>)")
    parser.add_argument("-f", "--fps", type=float, default=1, help="Frames per second to extract (default: 1)")
    parser.add_argument("-fmt", "--format", default="jpg", choices=["jpg", "png"], help="Output image format (default: jpg)")
    parser.add_argument("-q", "--quality", type=int, default=2, help="JPEG quality (1-31, 2=best, default: 2)")
    parser.add_argument("-p", "--prefix", default="frame", help="Filename prefix (default: frame)")

    args = parser.parse_args()

    # Validate input file exists
    if not os.path.isfile(args.video):
        print(f"Error: Video file '{args.video}' not found.")
        return

    # Generate output directory with filename if not provided
    if args.output is None:
        # Get the filename without extension
        filename = os.path.splitext(os.path.basename(args.video))[0]
        args.output = f"frames_{filename}"

    extract_frames(
        video_path=args.video,
        output_dir=args.output,
        fps=args.fps,
        format=args.format,
        quality=args.quality,
        prefix=args.prefix
    )

if __name__ == "__main__":
    main()

These are different operation that can be used when capturing the video

In [ ]:
# Basic usage (equivalent to your original command)
python extract_frames.py coffee_video.mp4

# Extract at 2 frames per second
python extract_frames.py coffee_video.mp4 -f 2

# Use PNG format for lossless quality
python extract_frames.py coffee_video.mp4 -fmt png

# Custom output directory and filename prefix
python extract_frames.py coffee_video.mp4 -o coffee_frames -p coffee

# Higher JPEG quality (slower, larger files)
python extract_frames.py coffee_video.mp4 -q 2

Additional features this script provides:
1. Error handling: Checks if the input file exists and if FFmpeg is installed

2. Flexible output options: Choose between JPG/PNG formats, adjust quality, and customize filenames

3. Directory creation: Automatically creates the output directory if it doesn't exist

4. Progress feedback: Shows status messages during extraction

5. Frame counting: Reports how many frames were extracted

6. Configurable extraction rate: Easily change how many frames per second to extract

For even more advanced functionality, you could extend this script to:
1. Extract frames only from a specific time range

2. Resize frames during extraction

3. Add timestamp overlays to each frame

4. Create a video preview alongside the frames

5. Generate a manifest file with frame metadata

# Below is how to capture frames from LiveCamera

In [ ]:
import cv2
import os
import time
from datetime import datetime
from pathlib import Path

class VideoFrameExtractor:
    def __init__(self, output_dir="captured_frames", capture_interval=1.0,
                 resolution=(1280, 720), prefix="frame", format="jpg"):
        """
        Initialize the video frame extractor

        Args:
            output_dir (str): Directory to save captured frames
            capture_interval (float): Time interval between frame captures in seconds
            resolution (tuple): Camera resolution (width, height)
            prefix (str): Prefix for saved frame filenames
            format (str): Image format (jpg, png)
        """
        self.output_dir = output_dir
        self.capture_interval = capture_interval
        self.resolution = resolution
        self.prefix = prefix
        self.format = format
        self.cap = None

        # Create output directory if it doesn't exist
        Path(output_dir).mkdir(parents=True, exist_ok=True)

    def start_capture(self, duration=None, camera_index=0):
        """
        Start capturing frames from the camera

        Args:
            duration (float): Total capture duration in seconds (None for indefinite)
            camera_index (int): Camera device index
        """
        # Initialize video capture
        self.cap = cv2.VideoCapture(camera_index)
        if not self.cap.isOpened():
            print("Error: Could not open camera")
            return

        # Set camera resolution
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.resolution[0])
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.resolution[1])

        print(f"Starting video capture. Press 'q' to stop.")
        print(f"Frames will be saved to: {self.output_dir}")
        print(f"Capture interval: {self.capture_interval} seconds")

        frame_count = 0
        last_capture_time = time.time()
        start_time = time.time()

        try:
            while True:
                # Check if duration limit reached
                if duration and (time.time() - start_time) >= duration:
                    break

                # Read frame from camera
                ret, frame = self.cap.read()
                if not ret:
                    print("Error: Failed to capture frame")
                    break

                current_time = time.time()

                # Check if it's time to capture a frame
                if current_time - last_capture_time >= self.capture_interval:
                    # Generate filename with timestamp
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                    filename = f"{self.prefix}_{timestamp}.{self.format}"
                    filepath = os.path.join(self.output_dir, filename)

                    # Save frame
                    if self.format.lower() == "jpg":
                        cv2.imwrite(filepath, frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
                    else:
                        cv2.imwrite(filepath, frame)

                    print(f"Captured frame: {filename}")
                    frame_count += 1
                    last_capture_time = current_time

                # Display preview (optional)
                cv2.imshow('Frame Capture', frame)

                # Check for quit command
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        except KeyboardInterrupt:
            print("\nCapture interrupted by user")
        finally:
            # Clean up
            self.cap.release()
            cv2.destroyAllWindows()
            print(f"Capture completed. Saved {frame_count} frames.")

    def set_interval(self, interval):
        """Change the capture interval during operation"""
        self.capture_interval = interval
        print(f"Capture interval changed to {interval} seconds")

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Capture frames from webcam at regular intervals")
    parser.add_argument("-o", "--output", default="captured_frames", help="Output directory")
    parser.add_argument("-i", "--interval", type=float, default=0.3, help="Capture interval in seconds")
    parser.add_argument("-d", "--duration", type=float, help="Total capture duration in seconds")
    parser.add_argument("-r", "--resolution", nargs=2, type=int, default=[1280, 720],
                        help="Camera resolution (width height)")
    parser.add_argument("-p", "--prefix", default="frame", help="Filename prefix")
    parser.add_argument("-f", "--format", default="jpg", choices=["jpg", "png"], help="Image format")
    parser.add_argument("-c", "--camera", type=int, default=0, help="Camera index")

    args = parser.parse_args()

    # Create frame extractor
    extractor = VideoFrameExtractor(
        output_dir=args.output,
        capture_interval=args.interval,
        resolution=tuple(args.resolution),
        prefix=args.prefix,
        format=args.format
    )

    # Start capture
    extractor.start_capture(duration=args.duration, camera_index=args.camera)

if __name__ == "__main__":
    main()

Below is how to run the script

In [ ]:
# Basic usage (capture 1 frame per second until 'q' is pressed)
python VideoFrameExtractor.py


# Capture for 60 seconds at 2 frames per second
python VideoFrameExtractor -d 60 -i 2

# Custom output directory and prefix
python VideoFrameExtractor -o my_captures -p my_video

# Higher resolution and PNG format
python VideoFrameExtractor -r 1920 1080 -f png

# Use a different camera (e.g., external webcam)
python VideoFrameExtractor -c 1

Key Features
1. Real-time Capture: Uses OpenCV to capture video from your webcam

2. Configurable Interval: Set how often to capture frames (e.g., 1 frame per second)

3. Duration Limit: Optionally set a maximum capture time

4. Flexible Output: Choose output directory, filename prefix, and image format

5. Preview Window: See what's being captured in real-time

6. Timestamped Filenames: Each frame includes a timestamp for easy sorting

7. Multiple Camera Support: Specify which camera to use (0 for default)

Practical Applications
1. Time-lapse Creation: Capture frames at intervals to create a time-lapse video

2. Motion Detection: Use captured frames for motion detection algorithms

3. Data Collection: Gather training data for computer vision models

4. Security Monitoring: Capture frames for security or monitoring purposes

5. Process Documentation: Document processes or experiments with periodic captures

Advanced Enhancements You Could Add
1. Motion Trigger: Only capture frames when motion is detected

2. Auto-brightness Adjustment: Adjust camera settings based on lighting conditions

3. Remote Control: Add web interface for remote control of the capture process

4. Cloud Upload: Automatically upload captured frames to cloud storage

5. Annotations: Add timestamp or other annotations directly to the frames